In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import ndcg_score
import dask.dataframe as dd

In [ ]:
df_node= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Erdos_tgbn_2024/data/tgbn-genre_node_labels.csv")
df_node['ts']= pd.to_datetime(df_node['ts'], unit= 's') # timestamp to datetime
df_node['ts'] = pd.to_datetime(df_node['ts'].dt.strftime('%Y-%m-%d')) # dont need time
df_node['user_id']=df_node['user_id'].str.slice(5).astype('Int64') #convert user_id to integers
df_node

,ts,user_id,genre,weight
0,2005-02-15,54,chillout,0.015835
1,2005-02-15,54,female vocalist,0.015330
2,2005-02-15,54,downtempo,0.008128
3,2005-02-15,54,electronic,0.072162
4,2005-02-15,54,reggae,0.021465
...,...,...,...,...
2741930,2009-06-12,651,katy perry,0.004864
2741931,2009-06-12,724,bebop,0.041841
2741932,2009-06-12,724,jazz,0.958159
2741933,2009-06-12,802,Britney Spears,0.359504


In [ ]:
dt= df_node.groupby(['user_id','genre']).agg({'weight': 'last'}).reset_index()
dt

,user_id,genre,weight
0,1,2000s,0.006737
1,1,2009,0.006737
2,1,70s,0.094771
3,1,80s,0.400000
4,1,Avant-Garde,0.035887
...,...,...,...
122555,1000,trip hop,0.137402
122556,1000,turntablism,0.078441
122557,1000,underground hip-hop,0.010696
122558,1000,violin,0.009207


In [ ]:
dtt= dt.groupby(["user_id",'genre'])["weight"].first().unstack(fill_value=0).stack().reset_index()
dtt.columns = [*dtt.columns[:-1], 'weight']
dtt # note 972*513=499662

,user_id,genre,weight
0,1,00s,0.000000
1,1,1970s,0.000000
2,1,1980s,0.000000
3,1,1990s,0.000000
4,1,2000s,0.006737
...,...,...,...
499657,1000,westlife,0.000000
499658,1000,whitney houston,0.000000
499659,1000,world,0.096618
499660,1000,wu-tang,0.000000


# Prediction

In [ ]:
ds= df_node.groupby(["ts", "user_id",'genre'])["weight"].first().unstack(fill_value=0).stack().reset_index()
ds.columns = [*ds.columns[:-1], 'weight']
ds_test= ds.loc[datetime(2008,10,19)<=ds['ts']]

In [ ]:
ds_test

,ts,user_id,genre,weight
106761969,2008-10-19,1,00s,0.0
106761970,2008-10-19,1,1970s,0.0
106761971,2008-10-19,1,1980s,0.0
106761972,2008-10-19,1,1990s,0.0
106761973,2008-10-19,1,2000s,0.0
...,...,...,...,...
131580904,2009-06-12,802,westlife,0.0
131580905,2009-06-12,802,whitney houston,0.0
131580906,2009-06-12,802,world,0.0
131580907,2009-06-12,802,wu-tang,0.0


In [ ]:
dtt.to_csv('/content/drive/MyDrive/Colab Notebooks/Erdos_tgbn_2024/data/trainer.csv', index=False)
ds_test.to_csv('/content/drive/MyDrive/Colab Notebooks/Erdos_tgbn_2024/data/tester.csv', index=False)

In [ ]:
df1 = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/Erdos_tgbn_2024/data/trainer.csv')
df2 = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/Erdos_tgbn_2024/data/tester.csv')
df = dd.merge(df2, df1, how='left', on=['user_id','genre'])

In [ ]:
df_pred = df.compute()
df_pred

,ts,user_id,genre,weight_x,weight_y
0,2008-10-19,1,00s,0.0,0.000000
1,2008-10-19,1,1970s,0.0,0.000000
2,2008-10-19,1,1980s,0.0,0.000000
3,2008-10-19,1,1990s,0.0,0.000000
4,2008-10-19,1,2000s,0.0,0.006737
...,...,...,...,...,...
2254978,2009-06-12,802,westlife,0.0,0.000000
2254979,2009-06-12,802,whitney houston,0.0,0.000000
2254980,2009-06-12,802,world,0.0,0.000000
2254981,2009-06-12,802,wu-tang,0.0,0.000000


In [ ]:
ndcg_score(np.array(df_pred['weight_x']).reshape(-1,513),np.array(df_pred['weight_y']).reshape(-1,513), k=10)

0.21801101647094928